In [1]:
# Wonderful hide/show script from http://blog.nextgenetics.net/?e=102
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Toggle Code Visibility"></form>''')

# Accuracy of Canada's Open Data Inventory
## An informal analysis of the Government of Canada's ability to keep track of their data

Recently, I decided to spend a weekend afternoon browsing through the troves of information available through the Government of Canada's Open Data site (a fairly common passtime for the average well-adjusted citizen, I would imagine). However, through making the mistake of browsing the Open Data *Inventory* instead of the Open Data *Portal*, I made an interesting observation: **about half of the datasets I tried to access were unavailable**. What's the difference? Well, according to the Government of Canada's [interactive version](https://open.canada.ca/en/search/inventory) of the Open Data Inventory and the [page for the underlying dataset](https://open.canada.ca/data/en/dataset/4ed351cf-95d8-4c10-97ac-6b3511f359b7):
> The Directive on Open Government requires all applicable Government of Canada organizations to conduct an inventory of their open data holdings.

> Departments were provided with an open data inventory template with standardized elements to populate, and upload in the metadata catalogue, the Open Government Registry. These elements are described in the data dictionary file.
Departments are responsible for maintaining up-to-date data inventories that reflect significant additions to their data holdings.

Essentially, the inventory is a compilation of submissions from each department, each describing that department's data holdings. It is entirely human-maintained. In comparison, the portal is a frontend for the stoage system almost all these datasets are stored on ([CKAN 2.6](https://docs.ckan.org/en/2.6/)). Listings on the portal are automated based on which datasets *actually exist* and are publicly viewable. This analysis is based on the **inventory**, and how well it manages to accurately point to up-to-date and available records on CKAN and other systems.

## Graph visability warning
I used Plotly to generate all my graphs, which requires iframes and Javascript to be enabled. This means viewing this notebook on Github will not work. Either view it through nbviewer or your own Jupyter instance.

## About this analysis
This is not a proper report. This is not a formal call for data management reform. This is just a casual peek at how well our federal government can keep a spreadsheet of where all their spreadsheets are. Don't expect any sort of regression or even a pearson correlation coefficient. You'll be lucky if I even tell you the standard deviation of user votes <sup><sup>(it may or may not be 2.546948)</sup></sup>. The code is messy, my web scraper died partway through (multithreading in iPython is an adventure), and I don't have any real insights that the GoC doesn't already have. I'm also clearly not an expert in government matters, and I could absolutely be wrong with any of my assertions about how the GoC works. This notebook was also very much written in a sequential, exploratory, almost stream-of-consciousness style. Expect lots of redundant code and inefficient methodology.

I started by trying to perform an HTTP GET request on every line of "portal_url_en" from the original inventory CSV that was marked as ready for public release. I then generated a new "availability" column from the results, with 3 possible states:

| Value     | Description                                                                                             |
|-----------|---------------------------------------------------------------------------------------------------------|
| available | There appears to be a valid data set at the portal url (returned HTTP 200 with no "deleted" page        |
| missing   | The URL points to a record that has been deleted or moved, such as a "Dataset Deleted" page or 404 page |
| invalid   | The URL is missing or cannot be properly resolved                                                       |

I also store the HTTP status number and HTML page title in their own columns (when applicable). This is all in "Open_Inventory_Merged.csv", which this notebook is built around visualizing. I want to stress, again, that when I mark a resource as "missing" or "invalid", this does **not** mean it is actually purged off the face of the Earth. It's almost certainly readily available on the portal with a different URL. This is *purely* an exercise in measuring the GoC's ability to keep their records up to date.

So, without further ado, let's look at that data, shall we?

The first thing we're doing is loading the CSV file into memory as a dataframe. Here's the first few rows, just so you get an idea of what we're looking at:

In [2]:
import pandas as pd, numpy as np, re, plotly.plotly as py, plotly.graph_objs as go
# py.init_notebook_mode(connected=True)
inventory = pd.read_csv("Open_Inventory_Merged.csv", encoding='utf-8')
inventory.head()

,Unnamed: 0,ref_number,title_en,title_fr,description_en,description_fr,publisher_en,publisher_fr,date_published,language,...,portal_url_en,portal_url_fr,user_votes,owner_org,owner_org_title,Unnamed: 0.1,url,http_status,page_title,availability
0,0,ODI-2018-01035,The AAFC Productivity Account for Canadian Agr...,Compte de productivité pour l’agriculture cana...,The AAFC Productivity Account for Canadian agr...,Le Compte de productivité pour l’agriculture c...,Agriculture and Agri-Food Canada,NaN,2018-08-09,en+fr,...,https://open.canada.ca/data/en/dataset/6adaeb1...,https://ouvert.canada.ca/data/fr/dataset/6adae...,0,aafc-aac,Agriculture and Agri-Food Canada | Agriculture...,0.0,https://open.canada.ca/data/en/dataset/6adaeb1...,200.0,The AAFC Productivity Account for Canadian Agr...,available
1,1,ODI-2018-01037,Swift Current Water Chemistry - Long Term Till...,Chimie de l'eau de Swift Current - Étude de tr...,The Swift current water quantity and quality i...,La quantité et la qualité de l'eau actuelle de...,Agriculture and Agri-Food Canada,Agriculture and Agri-Food Canada,2018-11-23,en+fr,...,https://open.canada.ca/data/en/dataset/b22cd29...,https://ouvert.canada.ca/data/fr/dataset/b22cd...,0,aafc-aac,Agriculture and Agri-Food Canada | Agriculture...,1.0,https://open.canada.ca/data/en/dataset/b22cd29...,200.0,Swift Current Water Chemistry - Long Term Till...,available
2,2,ODI-2018-01022,Minor Use Pesticides Program Project Status by...,Programme de pesticides à usage limité État d'...,The Pest Management Centre’s (PMC) Minor Use P...,Le Programme des pesticides à usage limité du ...,Agriculture and Agri-Food Canada,Agriculture and Agri-Food Canada,2018-12-14,en+fr,...,https://open.canada.ca/data/en/dataset/97addbb...,https://ouvert.canada.ca/data/fr/dataset/97add...,0,aafc-aac,Agriculture and Agri-Food Canada | Agriculture...,2.0,https://open.canada.ca/data/en/dataset/97addbb...,200.0,Minor Use Pesticides Program Project Status by...,available
3,3,ODI-2018-01023,Minor Use Pesticides Program Submission Status...,État des présentations de programmes de pestic...,The Pest Management Centre Minor Use Pesticide...,Le Programme des pesticides à usage limité du ...,Agriculture and Agri-Food Canada,Agriculture and Agri-Food Canada,2018-12-14,en+fr,...,https://open.canada.ca/data/en/dataset/97addbb...,https://ouvert.canada.ca/data/fr/dataset/97add...,0,aafc-aac,Agriculture and Agri-Food Canada | Agriculture...,2.0,https://open.canada.ca/data/en/dataset/97addbb...,200.0,Minor Use Pesticides Program Project Status by...,available
4,4,ODI-2018-01024,Minor Use Pesticides Program Trial Statistics ...,Statistiques d'essais du Programme de pesticid...,The following table provides information perta...,Le tableau suivant présente de l'information c...,Agriculture and Agri-Food Canada,Agriculture and Agri-Food Canada,2018-12-14,en+fr,...,https://open.canada.ca/data/en/dataset/97addbb...,https://ouvert.canada.ca/data/fr/dataset/97add...,0,aafc-aac,Agriculture and Agri-Food Canada | Agriculture...,2.0,https://open.canada.ca/data/en/dataset/97addbb...,200.0,Minor Use Pesticides Program Project Status by...,available


So how many URLs in the inventory are actually valid? Let's do a quick count...

In [3]:
availability_totals = inventory.groupby('availability').size()
availability_totals

availability
available    3119
invalid      2343
missing      5352
dtype: int64

Oh. Huh. That's... not great. Perhaps if I put it in a pie graph, I'll feel better?

In [4]:
total_pie_colors = ['#4caf50', '#f44336', '#ff9800']
total_pie_figure ={
    'data': [{
        'labels': availability_totals.index,
        'values': availability_totals.values,
        'marker': dict(colors=total_pie_colors),
        'type': 'pie'}],
    'layout': {'title': 'Availability Status of Resources Pointed to by Portal URLs'}}

py.iplot(total_pie_figure, filename='total_availability_breakdown')

Nope. I feel worse. Our slice of tasty data-filled lime pie is not nearly as big as that mysterious slice of orange pie. Even the erroneous strawberry pie slice is nearly as big as our poor key-value lime pie slice. Let's see who's responsible for bringing all this stale pie to our pie party...

## Which departments are to blame?
The first step is to create a simple boolean column of whether or not a URL is available. We don't care why it is or isn't available, we just care whether we can get that dataset.

In [5]:
inventory['is_available'] = np.where(inventory['availability'] == 'available', 1, 0)
inventory.head()

,Unnamed: 0,ref_number,title_en,title_fr,description_en,description_fr,publisher_en,publisher_fr,date_published,language,...,portal_url_fr,user_votes,owner_org,owner_org_title,Unnamed: 0.1,url,http_status,page_title,availability,is_available
0,0,ODI-2018-01035,The AAFC Productivity Account for Canadian Agr...,Compte de productivité pour l’agriculture cana...,The AAFC Productivity Account for Canadian agr...,Le Compte de productivité pour l’agriculture c...,Agriculture and Agri-Food Canada,NaN,2018-08-09,en+fr,...,https://ouvert.canada.ca/data/fr/dataset/6adae...,0,aafc-aac,Agriculture and Agri-Food Canada | Agriculture...,0.0,https://open.canada.ca/data/en/dataset/6adaeb1...,200.0,The AAFC Productivity Account for Canadian Agr...,available,1
1,1,ODI-2018-01037,Swift Current Water Chemistry - Long Term Till...,Chimie de l'eau de Swift Current - Étude de tr...,The Swift current water quantity and quality i...,La quantité et la qualité de l'eau actuelle de...,Agriculture and Agri-Food Canada,Agriculture and Agri-Food Canada,2018-11-23,en+fr,...,https://ouvert.canada.ca/data/fr/dataset/b22cd...,0,aafc-aac,Agriculture and Agri-Food Canada | Agriculture...,1.0,https://open.canada.ca/data/en/dataset/b22cd29...,200.0,Swift Current Water Chemistry - Long Term Till...,available,1
2,2,ODI-2018-01022,Minor Use Pesticides Program Project Status by...,Programme de pesticides à usage limité État d'...,The Pest Management Centre’s (PMC) Minor Use P...,Le Programme des pesticides à usage limité du ...,Agriculture and Agri-Food Canada,Agriculture and Agri-Food Canada,2018-12-14,en+fr,...,https://ouvert.canada.ca/data/fr/dataset/97add...,0,aafc-aac,Agriculture and Agri-Food Canada | Agriculture...,2.0,https://open.canada.ca/data/en/dataset/97addbb...,200.0,Minor Use Pesticides Program Project Status by...,available,1
3,3,ODI-2018-01023,Minor Use Pesticides Program Submission Status...,État des présentations de programmes de pestic...,The Pest Management Centre Minor Use Pesticide...,Le Programme des pesticides à usage limité du ...,Agriculture and Agri-Food Canada,Agriculture and Agri-Food Canada,2018-12-14,en+fr,...,https://ouvert.canada.ca/data/fr/dataset/97add...,0,aafc-aac,Agriculture and Agri-Food Canada | Agriculture...,2.0,https://open.canada.ca/data/en/dataset/97addbb...,200.0,Minor Use Pesticides Program Project Status by...,available,1
4,4,ODI-2018-01024,Minor Use Pesticides Program Trial Statistics ...,Statistiques d'essais du Programme de pesticid...,The following table provides information perta...,Le tableau suivant présente de l'information c...,Agriculture and Agri-Food Canada,Agriculture and Agri-Food Canada,2018-12-14,en+fr,...,https://ouvert.canada.ca/data/fr/dataset/97add...,0,aafc-aac,Agriculture and Agri-Food Canada | Agriculture...,2.0,https://open.canada.ca/data/en/dataset/97addbb...,200.0,Minor Use Pesticides Program Project Status by...,available,1


Now that we have that as a simple 1 or 0 value, we can do math stuff! In this case, math stuff mostly just means grouping by organization and calculating a mean for each one.

In [6]:
org_groups = inventory.groupby(['owner_org', 'owner_org_title'])
availability_by_org = org_groups['is_available'].mean().sort_values(ascending=False)*100
availability_by_org = availability_by_org.to_frame()
availability_by_org.reset_index(inplace = True)

Now we can make a quick bar graph to see which government departments are keeping their inventories up to date, and which ones are slacking off:

In [7]:
availability_by_org_rounded_percents = round(availability_by_org['is_available'], 2)
availability_by_org_hover = availability_by_org['owner_org_title'] + ' | ' + availability_by_org_rounded_percents.astype(str) + '%'

availability_by_org_data = [go.Bar(
    y = availability_by_org['owner_org'],
    x = availability_by_org['is_available'],
    orientation = 'h',
    text = availability_by_org_hover,
    hoverinfo = 'text',
    marker = {
        'color': 100-availability_by_org['is_available'],
        'colorscale': 'RdBu'
    }
)]

availability_by_org_layout = go.Layout(
    title='Availability Percentage by Organization',
    xaxis=dict(
        title='Percentage of Linked Resources Available'
    ),
    yaxis=dict(
        title='Government Organization'
    ),
    height=1000,
    margin=go.layout.Margin(
        l=150,
        r=10,
        b=60,
        t=30,
        pad=8
    )
)

availability_by_org_fig = go.Figure(data=availability_by_org_data, layout=availability_by_org_layout)

py.iplot(availability_by_org_fig, filename='org_availability_bar')

As you can see, there's a fairly wide variance of record availability. Some departments, such as the Courts Administration Service and Canada Border Services Angency, have perfect availability rates (100%). The CRA (97%) and DoJ (95%) are doing pretty well too. CRA, Border Services, Courts Administration, keep up the great work. However, a lot of departments such as the Transportation Safety Board (33%), Canadian Food Inspection Agency (32%), and Transport Canada (10%) are incredibly lacking in terms of how up-to-date their portal URLs are, and some departments, such as the Canadian Nuclear Safety Commission, National Film Board, and Global Affairs Canada, do not have a single URL that is actually usable.

## Number of Records vs Availability
You may be wondering if there is any correlation between how many resources an organization has to maintain vs how many of those resources are missing. Let's quickly compare the percentage of available records vs the total number of reported records...

First, we slap these two columns into a single dataframe:

In [8]:
quantity_vs_availability = availability_by_org.copy()
quantity_vs_availability.rename(columns={'is_available':'available_percent'}, inplace=True)
quantity_vs_availability = quantity_vs_availability.merge(
    org_groups.size().to_frame(),
    on='owner_org')
quantity_vs_availability.rename(columns={0:'portal_count'}, inplace=True)
quantity_vs_availability.head(10)

,owner_org,owner_org_title,available_percent,portal_count
0,wd-deo,Western Economic Diversification Canada | Dive...,100.000000,12
1,cas-satj,Courts Administration Service | Service admini...,100.000000,2
2,cbsa-asfc,Canada Border Services Agency | Agence des ser...,100.000000,10
3,infc,Infrastructure Canada | Infrastructure Canada,100.000000,27
4,nrc-cnrc,National Research Council Canada | Conseil nat...,100.000000,23
5,cra-arc,Canada Revenue Agency | Agence du revenu du Ca...,96.875000,96
6,lac-bac,Library and Archives Canada | Bibliothèque et ...,96.261682,107
7,jus,Department of Justice | Ministère de la Justice,95.238095,21
8,vac-acc,Veterans Affairs Canada | Anciens Combattants ...,94.117647,17
9,cic,"Immigration, Refugees and Citizenship Canada |...",89.237668,223


And now we can see these values on a scatter plot (with a logarithmic scale for the number of records, so we can see things more clearly): 

In [9]:
availability_by_portals_rounded_percents = round(quantity_vs_availability['available_percent'], 2)
availability_by_portals_hover = quantity_vs_availability['owner_org_title'] + ' | ' + availability_by_portals_rounded_percents.astype(str) + '% | ' + quantity_vs_availability['portal_count'].astype(str)

availability_by_portals_data = [go.Scatter(
    y = quantity_vs_availability['available_percent'],
    x = quantity_vs_availability['portal_count'],
    mode = 'markers',
    text = availability_by_portals_hover,
    hoverinfo = 'text',
    marker = {
        'color': 100-quantity_vs_availability['available_percent'],
        'colorscale': 'RdBu',
        'size': 12
    }
)]

availability_by_portals_layout = go.Layout(
    title='Availability Percentage per Organization by Number of Resources',
    xaxis=dict(
        type='log',
        title='Percentage of Linked Resources Available'
    ),
    yaxis=dict(
        title='Number of Resources Owned by Organization (Log Scale)'
    )
)

availability_by_portals_fig = go.Figure(data=availability_by_portals_data, layout=availability_by_portals_layout)

py.iplot(availability_by_portals_fig, filename='org_availability_by_quantity')

...Huh. As it turns out, there's not really much of a correlation between how many records a department has to maintain and how well they maintain their part of the inventory. Maybe user votes have some sort of influence on how up to date the records are?

## User Votes vs Availability

This gets a little harder to neatly visualise, since we're plotting a boolean over a range of integers. Let's start by looking at an average availability rate vs number of votes...

In [10]:
availability_by_votes = inventory[['user_votes','is_available']] \
                        .groupby('user_votes').mean() \
                        .join(
                            inventory[['user_votes','is_available']]
                            .groupby('user_votes').size().to_frame(),
                            on='user_votes') \
                        .rename(columns={0:'count'})

availability_by_votes

,is_available,count
user_votes,,
0,0.419899,2372
1,0.258102,6048
2,0.223932,1170
3,0.213894,547
4,0.278746,287
5,0.262774,137
6,0.227273,66
7,0.196429,56
8,0.321429,28


Interesting. Let's see that plotted out...

In [11]:
availability_by_votes_binned = availability_by_votes.copy().reset_index()
availability_by_votes_binned['user_votes'] = (availability_by_votes_binned['user_votes']//5)*5
availability_by_votes_binned = availability_by_votes_binned.groupby('user_votes').mean()
availability_by_votes_binned

,is_available,count
user_votes,,
0,0.278914,2084.800000
5,0.259914,62.200000
10,0.275000,8.400000
15,0.500000,1.750000
20,0.500000,2.000000
25,0.166667,3.250000
30,0.000000,1.000000
35,0.500000,1.500000
40,0.000000,1.000000


In [12]:
availability_by_votes_trace_points = go.Scatter(
    x = availability_by_votes.index.values,
    y = availability_by_votes['is_available'],
    mode = 'markers',
    marker = {
        'size': (np.log(availability_by_votes['count'])*2)+10
    },
    name = 'Availability Percentage'
)

availability_by_votes_trace_curve = go.Scatter(
    x = availability_by_votes_binned.index.values,
    y = availability_by_votes_binned['is_available'],
    mode = 'lines',
    line = {
        'shape': 'spline'
    },
    name = "World's Worst Attempt at Curve Fitting"
)

availability_by_votes_layout = go.Layout(
    title='Availability Percentage vs User Votes',
    xaxis=dict(
        title='Number of User Votes'
    ),
    yaxis=dict(
        title='Percentage of Resources Available'
    )
)

availability_by_votes_data = [availability_by_votes_trace_points, availability_by_votes_trace_curve]

py.iplot(go.Figure(availability_by_votes_data, availability_by_votes_layout), filename='availability_by_votes')

Another dead end. I even tried fitting a curve to the scatter plot by binning to the nearest 5 votes and averaging, and I made it all pretty by setting the display type to spline, but ultimately there's just not enough data to get any meaningful insights from user votes. Turns out not many people are voting on which datasets they care about.